In [1]:
import carla
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import learn2learn as l2l
import csv
import time

import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(NeuralNetwork, self).__init__()

        self.l1 = nn.Linear(state_dim, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = F.relu(self.l1(state))
        x = F.relu(self.l2(x))
        #action_logits = self.l3(x)
        actions = self.l3(x)
        #ction_probs = F.softmax(action_logits, dim=1)

        return actions


class CarlaEnvironment:
    def __init__(self, world, vehicle_blueprint, waypoints):
        self.world = world
        self.vehicle_blueprint = vehicle_blueprint
        self.waypoints = waypoints
        self.actorlist = []
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))
        
    def reset(self):
        self.vehicle.destroy()
        self.vehicle = self.world.spawn_actor(self.vehicle_blueprint, carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)))  
        self.current_waypoint = 0
        return self.get_state()

    def step(self, action):
        action = action.detach().numpy()
        #print(action)
        # Apply actions: Throttle, Steering, Brake
        self.vehicle.apply_control(carla.VehicleControl(throttle = float(action[0]) , steer = float(action[1]) , brake = float(action[2])))
       
        #self.actorlist.append(self.vehicle)

        # Get next state
        next_state = self.get_state()

        # Calculate reward
        reward = self.calculate_reward(next_state) 

        # Check if the episode is done
        done = self.is_done(next_state)

        return next_state, reward, done

#     def get_state(self):
#         curr_vel = self.vehicle.get_velocity()
#         curr_yaw = self.vehicle.get_transform().rotation.yaw
#         waypoint = self.waypoints[self.current_waypoint]
#         return np.array([curr_vel, curr_yaw - waypoint[1], curr_yaw - waypoint[2], waypoint[3]])
    
    def get_state(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw
        
        
        for i in range(10):
            if self.current_waypoint + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                vx = curr_vel.x
                vy = curr_vel.y
                v = ((vx)**2+(vy)**2)**0.5
                distance = curr_loc.distance(carla.Location(x=waypoint[4],y=waypoint[5]))
                state.extend([v, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
            else:
                state.extend([0, 0, 0, 0])
        return np.array(state)

    def generate_waypoints(self):
        # Implement waypoint generation logic
        pass

    def calculate_reward(self, state):
        reward = -(np.linalg.norm(state[0] - self.waypoints[self.current_waypoint][0]) - (np.linalg.norm (state[1])) - (np.linalg.norm ( state[2])) - (np.linalg.norm (state[3])) + self.current_waypoint)
        return reward

    def is_done(self, state):
        if np.abs(state[3]) < 0.20:  # If the vehicle is close to the next waypoint
            self.current_waypoint += 1

        if self.current_waypoint >= len(self.waypoints):
            return True

        return False

    def set_weather(self, weather):
        self.world.set_weather(weather)
        
    def destroy(self):
        for actor in self.actorlist:
            actor.destroy()
        print("All Cleared")


def train_ppo_maml(env, state_dim, action_dim, lr=1e-4, n_iters=1000000, inner_loop_steps=5):
    
 
    p_low = 0
    p_mid = 50
    p_high = 100

    pd_low = 50
    pd_high = 100

    tf_low = 0.5
    tf_mid = 1.5
    tf_high = 2.5
    
    conditions = [[p_low, pd_low, tf_high],
            [p_low, pd_low, tf_mid],
            [p_low, pd_high, tf_low],
            [p_mid, pd_low, tf_low],
            [p_mid, pd_high, tf_low],
            [p_mid, pd_low, tf_mid],
            [p_high, pd_low, tf_high],
            [p_high, pd_high, tf_low],
            [p_low, pd_high, tf_mid],
            [p_high, pd_low, tf_mid]]
    
    n_tasks = len(conditions)

    
    base_model = NeuralNetwork(state_dim, action_dim)
    maml = l2l.algorithms.MAML(base_model, lr = lr)
    optimizer = optim.Adam(maml.parameters(), lr = lr)
    iteration = 1
    for condition in conditions:
        
        precipitation = condition[0]
        precipitation_deposits = condition[1]
        tire_friction = condition[2]
        weather = carla.WeatherParameters( precipitation = precipitation,
                                            precipitation_deposits = precipitation_deposits)
        world.set_weather(weather)

        tire_condition = carla.WheelPhysicsControl(tire_friction = tire_friction)

        front_left_wheel  = carla.WheelPhysicsControl(tire_friction = tire_friction)
        front_right_wheel = carla.WheelPhysicsControl(tire_friction = tire_friction)
        rear_left_wheel   = carla.WheelPhysicsControl(tire_friction = tire_friction)
        rear_right_wheel  = carla.WheelPhysicsControl(tire_friction = tire_friction)

        wheels = [front_left_wheel, front_right_wheel, rear_left_wheel, rear_right_wheel]
        
        physics_control = env.vehicle.get_physics_control()
        physics_control.wheels = wheels
        env.vehicle.apply_physics_control(physics_control)
    
        trajectories, state_tensors, action_tensors = sample_trajectories(env, maml)
        state_tensors_stacked = torch.stack(state_tensors)
        action_tensors_stacked = torch.stack(action_tensors)
        
        for _ in range(inner_loop_steps):
        
            learner = maml.clone()
            error = nn.MSELoss()(learner(state_tensors_stacked), action_tensors_stacked)
            learner.adapt(error)
            error = nn.MSELoss()(learner(state_tensors_stacked), action_tensors_stacked)
            error.backward(retain_graph=True)

            
        optimizer.zero_grad()
        optimizer.step()
        iteration += 1
        print("Iteration for {condition}: ", iteration)

        if iteration % 1000 == 0:
            print("Iteration for {condition}: ", iteration)


def sample_weather_conditions(num_weather_conditions):
    return carla.WeatherParameters.HardRainNoon

def sample_trajectories(env, model, num_episodes=10, max_episode_length=1000):
#     model.to("cpu")
    trajectories = []
    traj_states = []
    traj_action = []
    state_tensors = []
    action_tensors = []
       
    for _ in range(num_episodes):
        state = env.reset()                
        trajectory = []

        for j in range(max_episode_length):
            
#             state_tensor = torch.tensor(state).to(torch.float32)
            state_tensor = torch.from_numpy(state.astype(np.float32))
            state_tensors.append(state_tensor)
            action_tensor = model(state_tensor)
#             action_distribution = Normal(action_mean, torch.tensor(0.1))
#             action = action_distribution.sample().detach().numpy().astype(float)
            
            action = action_tensor
#             action_tensor = torch.tensor(action, dtype=torch.float32)
            action_tensors.append(action)

            next_state, reward, done = env.step(action)
            trajectory.append((state, action, reward, next_state, done))

            state = next_state
            
            if not j%200 : print("Reward: ", reward, '  J: ', j )
            #state_tensors = torch.tensor(state_tensor, dtype=torch.float32)
            if done or j==max_episode_length-1:
                #env.vehicle.destroy()
                break  

        trajectories.append(trajectory)
    return trajectories, state_tensors, action_tensors

            
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            v, yaw_c, yaw_n, r, x, y = map(float, row)
            waypoints.append((v, yaw_c, yaw_n, r, x, y))
            
    return waypoints                        

In [2]:
if __name__ == '__main__':

    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)

    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    vehicle_blueprint = blueprint_library.filter("wrangler_rubicon")[0]

    waypoints = read_waypoints("waypoint_with_xy.csv")
    env = CarlaEnvironment(world, vehicle_blueprint, waypoints)  # Implement CarlaEnvironment with the required state and action spaces

    state_dim = 40
    action_dim = 3
    train_ppo_maml(env, state_dim, action_dim)
    

    
        

Reward:  139.78680419921875   J:  0
Reward:  1.7909103631973267   J:  200
Reward:  1.720502495765686   J:  400
Reward:  1.553402841091156   J:  600
Reward:  1.418022871017456   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7444543838500977   J:  200
Reward:  1.6693082451820374   J:  400
Reward:  1.6296700835227966   J:  600
Reward:  1.5182350873947144   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7769770622253418   J:  200
Reward:  1.7193748950958252   J:  400
Reward:  1.6257873177528381   J:  600
Reward:  1.5058684349060059   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7515251636505127   J:  200
Reward:  1.6619996428489685   J:  400
Reward:  1.5368287563323975   J:  600
Reward:  1.4083020687103271   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7510809898376465   J:  200
Reward:  1.6862658858299255   J:  400
Reward:  1.5930089950561523   J:  600
Reward:  1.463488757610321   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.775291085243225  

Reward:  139.78680419921875   J:  0
Reward:  1.7731844186782837   J:  200
Reward:  1.7469037771224976   J:  400
Reward:  1.6676188111305237   J:  600
Reward:  1.4733890891075134   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7710598707199097   J:  200
Reward:  1.7126985788345337   J:  400
Reward:  1.6644995212554932   J:  600
Reward:  1.6079776287078857   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.768855094909668   J:  200
Reward:  1.7079108953475952   J:  400
Reward:  1.6590557098388672   J:  600
Reward:  1.5390364527702332   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7735605239868164   J:  200
Reward:  1.7145267724990845   J:  400
Reward:  1.6721057891845703   J:  600
Reward:  1.6179659962654114   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.768631100654602   J:  200
Reward:  1.7428185939788818   J:  400
Reward:  1.7048790454864502   J:  600
Reward:  1.598329246044159   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7734768390655518

Reward:  139.78680419921875   J:  0
Reward:  1.753617763519287   J:  200
Reward:  1.7078702449798584   J:  400
Reward:  1.6525784134864807   J:  600
Reward:  1.566231608390808   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7568917274475098   J:  200
Reward:  1.708061933517456   J:  400
Reward:  1.646802306175232   J:  600
Reward:  1.5603755116462708   J:  800
Iteration for {condition}:  10
Reward:  139.78680419921875   J:  0
Reward:  1.7759673595428467   J:  200
Reward:  1.7247108221054077   J:  400
Reward:  1.6744953393936157   J:  600
Reward:  1.6060082912445068   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.7767575979232788   J:  200
Reward:  1.725818157196045   J:  400
Reward:  1.676698625087738   J:  600
Reward:  1.585311233997345   J:  800
Reward:  139.78680419921875   J:  0
Reward:  1.775455355644226   J:  200
Reward:  1.726457953453064   J:  400
Reward:  1.6788659691810608   J:  600
Reward:  1.6084001660346985   J:  800
Reward:  139.78680419921875   J:  0
Re